In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import riskfolio as rp
import math
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy.optimize import minimize
import yfinance as yf
from marketdatalib import map_scrip_to_yfin_ticker, get_nifty_index_data, calculate_beta
import riskfolio as rp

In [ ]:
# Number of year of historic data to consider
YEARS_OF_HISTORY = 5

# The risk free return rate. This is often the return on a 10 year government bond.
# At present, the 10 year government bond rate is 6.84%. We are using 7.5% for the sake of simplicity which is the rate of a fixed deposit.
RISK_FREE_RETURN = 0.075
EXPECTED_MARKET_RETURN = 0.18
ACCEPTABLE_VOLATILITY = 0.15

# In intend to evaluate the performance of the portfolio, we will compare it with the performance of the benchmark index.
BENCHMARK_INDEX_NIFTY500 = 'NIFTY%20500'

scripts_to_ignore = ['SONACOMS']
special_scrips = ['ARE&M']

In [ ]:
# Get benchmark data
NIFTY500_DATA = get_nifty_index_data(index_name=BENCHMARK_INDEX_NIFTY500, number_of_years=YEARS_OF_HISTORY)
NIFTY500_DATA.drop(columns=['Open', 'High', 'Low'], inplace=True)
NIFTY500_DATA['NIFTY500_returns'] = NIFTY500_DATA['Close'].pct_change(fill_method=None)
NIFTY500_DATA['NIFTY500_log_returns'] = NIFTY500_DATA['Close'].apply(lambda x: math.log(x)).diff()
NIFTY500_DATA.dropna(inplace=True, axis=0)
NIFTY500_DATA.rename(columns={'Close': 'NIFTY500'}, inplace=True)
# NIFTY500_DATA[['Close', 'NIFTY500_returns', 'NIFTY500_log_returns']]
NIFTY500_DATA

Since I'll be running this exercise on my own portfolio, there are things that I need to hide before I commit the same to githud :-). So, assume that I have CSV of the stocks that I hold and I'll be using that to run this exercise.

In [ ]:
holdings = pd.read_csv('../../data/MarketData/holdings.csv', names={'Instrument': 'Instrument', 'Quantity': 'Qty.', 'AvgCost': 'Avg. cost', 'LTP': 'LTP', 'CurVal': 'Cur. val', 'P&L': 'P&L', 'NetChg_Pct': 'Net chg.', 'DayChange_Pct': 'Day chg.', '':''}, header=0, index_col=0)
holdings = holdings.loc[:, holdings.columns != '']

indices_to_remove = [idx for idx in scripts_to_ignore if idx in holdings.index]
holdings = holdings.drop(indices_to_remove, axis=0)

holdings['Weight'] = holdings['CurVal'] / holdings['CurVal'].sum()

portfolio_stocks = [map_scrip_to_yfin_ticker(scrip, 'NSE' if scrip not in special_scrips else 'BSE') for scrip in holdings.index]
holdings

In [ ]:
master_data = pd.DataFrame()

start_date = datetime.today() - relativedelta(years=YEARS_OF_HISTORY)

scrips = [scrip for scrip, ticker in portfolio_stocks]
assets = [ticker.ticker for _, ticker in portfolio_stocks]

master_data = yf.download(assets, start = start_date)
master_data = master_data.loc[:,('Adj Close', slice(None))]
master_data.columns = scrips
master_data.index = pd.to_datetime(master_data.index)
master_data = master_data.sort_index()
master_data

In [ ]:
def encich_with_returns(data: pd.DataFrame):
    for scrip, _ in portfolio_stocks:
        data[scrip + '_returns'] = data[scrip].pct_change(fill_method=None)
        data[scrip + '_log_returns'] = data[scrip].apply(lambda x: math.log(x)).diff()
    data.dropna(inplace=True, axis=0)
    return data

master_data = encich_with_returns(master_data)
master_data = pd.merge(master_data, NIFTY500_DATA, left_index=True, right_index=True)
master_data

At this point of time, we will compute a few portfolio specific statistics that are agnostic of weights of individual stocks in the poortfolio.
- Coovariance matrix

In [ ]:
covariance_matrix = master_data[[scrip + '_returns' for scrip in scrips]].cov().to_numpy()

In [ ]:
realized_returns = pd.DataFrame(index=[scrip for scrip, _ in portfolio_stocks], columns=['Returns', 'AnnualizedReturns', 'DailyVolatility', 'AnnualizedVolatility', 'SharpeRatio', 'SortinoRatio', 'JensensAlpha', 'Beta'])

market_std = master_data['NIFTY500_returns'].std()

for scrip in scrips:
    realized_returns.loc[scrip, 'Returns'] = (master_data[scrip + '_returns'] + 1).prod() - 1

    # Calculate average daily return
    avg_daily_return = np.mean(master_data[scrip + '_returns'])
    annualized_return = (1 + avg_daily_return).mean() ** 252 - 1
    # Calculate downside deviation
    downside_returns = master_data[scrip + '_returns'][master_data[scrip + '_returns'] < 0]
    downside_deviation = np.std(downside_returns)
    annualized_downside_deviation = downside_deviation * np.sqrt(252)
    daily_volatility = (master_data[scrip + '_returns']).std()
    annualized_volatility = daily_volatility * math.sqrt(252)
    beta = calculate_beta(master_data[scrip + '_returns'], master_data['NIFTY500_returns'])

    realized_returns.loc[scrip, 'DailyVolatility'] = daily_volatility
    annualized_std = realized_returns.loc[scrip, 'AnnualizedVolatility'] = annualized_volatility
    realized_returns.loc[scrip, 'AnnualizedReturns'] = annualized_return
    realized_returns.loc[scrip, 'SharpeRatio'] = (realized_returns.loc[scrip, 'AnnualizedReturns'] - RISK_FREE_RETURN) / annualized_std
    
    realized_returns.loc[scrip, 'Beta'] = beta
    realized_returns.loc[scrip, 'ExpectedReturn'] = RISK_FREE_RETURN + beta * (EXPECTED_MARKET_RETURN - RISK_FREE_RETURN)

    # Calculate Sortino Ratio
    sortino_ratio = (annualized_return - RISK_FREE_RETURN) / annualized_downside_deviation

    realized_returns.loc[scrip, 'SortinoRatio'] = sortino_ratio
    # 'JensensAlpha'
    # Calculate Jensen's Alpha
    realized_returns.loc[scrip, 'JensensAlpha'] = realized_returns.loc[scrip, 'AnnualizedReturns'] - (RISK_FREE_RETURN + beta * (EXPECTED_MARKET_RETURN - RISK_FREE_RETURN))

realized_returns

The above table represents the the statistics associated with each of the security in our portfolio. The benchmark index that we're using for calculations is the NIFTY 500 TRI index. The expected return above is based on the current weightage of each of the security in the portfolio.

- The expected return is calculated by multiplying the weightage of each security with the expected return of the security.
- The Sharpe ratio of the portfolio is calculated by dividing the expected return of the portfolio by the standard deviation of the portfolio. The Sharpe ratio of the portfolio is used to calculate the risk-adjusted return of the portfolio.
- The risk-adjusted return of the portfolio is calculated by multiplying the Sharpe ratio of the portfolio with the expected return of the portfolio. The risk-adjusted return of the portfolio is used to calculate the risk-adjusted return of each security. The risk-adjusted return of each security is calculated by multiplying the Sharpe ratio of the portfolio with the expected return of the security.

In [ ]:
portfolio_expected_return = holdings['Weight'].dot(realized_returns['ExpectedReturn'])

weights = holdings['Weight']

portfolio_variance = np.dot(weights.T, np.dot(covariance_matrix, weights))

# Calculate the portfolio standard deviation (volatility)
portfolio_std_dev = np.sqrt(portfolio_variance)

# Calculate the Sharpe Ratio
portfolio_sharpe_ratio = (portfolio_expected_return - RISK_FREE_RETURN) / portfolio_std_dev

print('Expected Portfolio Return: {0:.2f}%'.format(portfolio_expected_return * 100))
print('Expected Portfolio Volatility: {0:.2f}%'.format(portfolio_std_dev * 100))
print('Sharpe Ratio of Portfolio: {0:.2f}'.format(portfolio_sharpe_ratio))
print('Risk adjusted return of Portfolio: {0:.2f}'.format(portfolio_sharpe_ratio * portfolio_expected_return))

One extremely important factor that we need to keep in mind is that for the sake of simplicity and for lack of data, we've only considered the last 5 years of benchmark data. This is going to hide the fact that the market might have given an extremely high return in the last 5 years (specially after COVID) and the expected return of the portfolio might not be replicable in the coming future. This is a limitation of the model and we need to keep this in mind while making any investment decisions based on the model.

This is also the reason why we've considered an error of 5% in the expected return of the portfolio. i.e., we are assuming that the expected rate of return of the portfolio can be 5% lower than the expected rate of return that we've calculated.

# Portfolio Optimization

In the next section, we are going to try and optimize the portfolio. The purpose of this exercise is basically to find the optimal weights of each security in the portfolio such that the portfolio has the highest Sharpe ratio. i.e., maximize risk adjusted returns foor a given level of risk.

I'm going to make use of a library called riskfolio-lib to optimize the portfolio.

In [ ]:
# Step 1: Define the expected returns and volatilities for each security
# correlation_matrix = master_data[[scrip + '_returns' for scrip, _ in portfolio_stocks]].corr().to_numpy()
securities = [scrip for scrip in scrips]
expected_returns = realized_returns['ExpectedReturn'].to_numpy()
volatilities = realized_returns['DailyVolatility'].to_numpy()

# Step 2: Calculate the covariance matrix
cov_matrix = covariance_matrix
# cov_matrix = np.outer(volatilities, volatilities) * correlation_matrix

# Step 3: Generate random portfolios
num_portfolios = 1000
results = np.zeros((3, num_portfolios))

for i in range(num_portfolios):
    weights = np.random.random(len(securities))
    weights /= np.sum(weights)
    
    portfolio_return = np.dot(weights, expected_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    
    results[0,i] = portfolio_return
    results[1,i] = portfolio_volatility
    results[2,i] = results[0,i] / results[1,i]  # Sharpe Ratio

# Step 4: Plot the efficient frontier
plt.figure(figsize=(12, 10))
plt.scatter(results[1,:], results[0,:], c=results[2,:], cmap='viridis')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')
plt.title('Efficient Frontier')
plt.show()

Next, I'll compute the capital allocation line

In [ ]:
# Define the number of securities
num_securities = len(securities)

# Define the objective function to minimize (negative Sharpe ratio)
def objective(weights):
    portfolio_return = np.dot(weights, expected_returns)
    portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = portfolio_return / portfolio_volatility
    return -sharpe_ratio

# Define the constraint for acceptable volatility
def volatility_constraint(weights):
    return ACCEPTABLE_VOLATILITY - np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))

# Define the constraints and bounds
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
               {'type': 'ineq', 'fun': volatility_constraint})
bounds = tuple((0, 1) for _ in range(num_securities))

# Initial guess for the weights
initial_guess = num_securities * [1. / num_securities]

# Perform the optimization
result = minimize(objective, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)

# Get the optimal weights
optimal_weights = result.x

# Calculate the expected return and volatility of the optimal risky portfolio
optimal_portfolio_return = np.dot(optimal_weights, expected_returns)
optimal_portfolio_volatility = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))

# Generate the Capital Allocation Line (CAL)
cal_returns = []
cal_volatilities = []

for weight in np.linspace(0, 1.5, 100):
    cal_return = RISK_FREE_RETURN + weight * (optimal_portfolio_return - RISK_FREE_RETURN)
    cal_volatility = weight * optimal_portfolio_volatility
    cal_returns.append(cal_return)
    cal_volatilities.append(cal_volatility)

# Plot the CAL
plt.figure(figsize=(10, 6))
plt.plot(cal_volatilities, cal_returns, label='Capital Allocation Line (CAL)')
plt.scatter(optimal_portfolio_volatility, optimal_portfolio_return, color='red', marker='*', s=100, label='Optimal Risky Portfolio')
plt.xlabel('Volatility (Standard Deviation)')
plt.ylabel('Expected Return')
plt.title('Capital Allocation Line (CAL)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
for fund, pct_allocation in zip(scrips, optimal_weights):
    print('\t{0}: {1:.2f}%'.format(fund, pct_allocation * 100))
    
print('\n\nExpected Portfolio Return: {0:.2f}%'.format(np.dot(optimal_weights, expected_returns) * 100))
print('Expected Portfolio Volatility: {0:.2f}%'.format(optimal_portfolio_volatility * 100))

In [ ]:
# Plot the CAL and the efficient frontier
plt.figure(figsize=(10, 6))
plt.scatter(results[1, :], results[0, :], c=results[2, :], cmap='viridis', marker='o', s=10, alpha=0.3, label='Efficient Frontier')
plt.plot(cal_volatilities, cal_returns, label='Capital Allocation Line (CAL)', color='red')
plt.scatter(optimal_portfolio_volatility, optimal_portfolio_return, color='blue', marker='*', s=100, label='Optimal Risky Portfolio')
plt.xlabel('Volatility (Standard Deviation)')
plt.ylabel('Expected Return')
plt.title('Efficient Frontier and Capital Allocation Line (CAL)')
plt.colorbar(label='Sharpe Ratio')
plt.legend()
plt.grid(True)
plt.show()